### Heroes Of Pymoli Data Analysis
* Of the 1163 active players, the vast majority are male (84%). There also exists, a smaller, but notable proportion of female players (14%).

* Our peak age demographic falls between 20-24 (44.8%) with secondary groups falling between 15-19 (18.60%) and 25-29 (13.4%).  
-----

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
file_to_load = "Resources/purchase_data.csv"

# Read Purchasing File and store into Pandas data frame
purchase_data = pd.read_csv(file_to_load)

In [3]:
#Clean up data and return data type
purchase_data = purchase_data.dropna(how='any')
purchase_data.dtypes

Purchase ID      int64
SN              object
Age              int64
Gender          object
Item ID          int64
Item Name       object
Price          float64
dtype: object

In [4]:
#Display data frame
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [5]:
#Display columns
purchase_data.columns

Index(['Purchase ID', 'SN', 'Age', 'Gender', 'Item ID', 'Item Name', 'Price'], dtype='object')

## Player Count

* Display the total number of players


In [6]:
totalPlayers = purchase_data['SN'].nunique()
totalPlayers_df = pd.DataFrame({'Total Players': totalPlayers}, index=[0])
totalPlayers_df

,Total Players
0,576


## Purchasing Analysis (Total)

* Run basic calculations to obtain number of unique items, average price, etc.


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame


In [7]:
#Calculate items
unique_items = len(purchase_data['Item ID'].unique())
average_price = purchase_data['Price'].mean()
purchases = len(purchase_data['Item ID'])
total_revenue = purchase_data['Price'].sum()

#Return items to dictionary
results = {
    'Number of Unique Items': unique_items,
    'Average Price': average_price,
    'Number of Purchases': purchases,
    'Total Revenue': total_revenue}

#Convert dictionary to Data Frame
results_df = pd.DataFrame(results, index=[0])

#Format results to make it look better
for col in ['Average Price', 'Total Revenue']:
    results_df[col] = results_df[col].map('${:.2f}'.format)

#Print results
results_df


,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


## Gender Demographics

* Percentage and Count of Male Players


* Percentage and Count of Female Players


* Percentage and Count of Other / Non-Disclosed




In [8]:
#Make new data frame for genders based on value counts
genders_df = purchase_data.groupby('Gender')[['SN']].nunique()

#Change name of column to to Total Count
genders_df = genders_df.rename(columns={'SN':'Total Count'})

#Add new column Percentage of players
genders_df['Percentage of Players'] = round((genders_df['Total Count']/genders_df['Total Count'].sum())*100,2)

#Return result
genders_df

,Total Count,Percentage of Players
Gender,,
Female,81,14.06
Male,484,84.03
Other / Non-Disclosed,11,1.91



## Purchasing Analysis (Gender)

* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. by gender




* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [11]:
# Group by genders
genders_group = purchase_data.groupby('Gender')

# Data Frame for count and Total
genders_count = pd.DataFrame(genders_group.count())
genders_total = pd.DataFrame(genders_group.sum())

# Merge table to calculate values
genders_two = pd.merge(genders_count, genders_total, on='Gender', how='outer', suffixes=('_Count','_Total'))
genders_three = pd.merge(genders_two, genders_df, on='Gender', how='outer', suffixes=('_M','_G'))

#Create working data frame
genders_working = genders_three.loc[:,['Purchase ID_Count', 'Price_Total', 'Total Count']]
genders_working = genders_working.rename(
    columns={
        'Purchase ID_Count':'Purchase Count',
        'Price_Total': 'Total Purchase Value',
        'Total Count': 'Unique Players Count'})
genders_working

# Calculating result
genders_results = genders_working[['Purchase Count', 'Total Purchase Value']]
genders_results['Average Purchase Price'] = genders_working['Total Purchase Value']/genders_working['Purchase Count']
genders_results['Avg Total Purchase per Person'] = genders_working['Total Purchase Value']/genders_working['Unique Players Count']

# Formating results
for col in ['Total Purchase Value', 'Average Purchase Price', 'Avg Total Purchase per Person']:
    genders_results[col] = genders_results[col].map('${:.2f}'.format)

orders = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Avg Total Purchase per Person']

genders_results[orders]

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


## Age Demographics

* Establish bins for ages


* Categorize the existing players using the age bins. Hint: use pd.cut()


* Calculate the numbers and percentages by age group


* Create a summary data frame to hold the results


* Optional: round the percentage column to two decimal points


* Display Age Demographics Table


In [21]:
# Create bins and group
bins = [0, 9, 14, 19, 24, 29, 34, 39,100]
groups = ['<10', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39', '40+']

# Add Age Group to data
purchase_data['Age Group'] = pd.cut(purchase_data['Age'], bins, labels=groups)

# Summarize Age Group
age_df = purchase_data.groupby('Age Group')[['SN']].nunique()
age_results = age_df.rename(columns={'SN':'Total Count'})
age_results['Percentage of Players'] = round(age_results['Total Count']/totalPlayers*100,2)
age_results


,Total Count,Percentage of Players
Age Group,,
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


## Purchasing Analysis (Age)

* Bin the purchase_data data frame by age


* Run basic calculations to obtain purchase count, avg. purchase price, avg. purchase total per person etc. in the table below


* Create a summary data frame to hold the results


* Optional: give the displayed data cleaner formatting


* Display the summary data frame

In [31]:
# Group by age
age_group = purchase_data.groupby('Age Group')

# Data Frame for count and Total
age_count = pd.DataFrame(age_group.count())
age_total = pd.DataFrame(age_group.sum())

# Merge table to calculate values
age_two = pd.merge(age_count, age_total, on='Age Group', how='outer', suffixes=('_Count','_Total'))
age_three = pd.merge(age_two, age_results, on='Age Group', how='outer', suffixes=('_M','_A'))

#Create working data frame
age_working = age_three.loc[:,['Purchase ID_Count', 'Price_Total', 'Total Count']]
age_working = age_working.rename(
    columns={
        'Purchase ID_Count':'Purchase Count',
        'Price_Total': 'Total Purchase Value',
        'Total Count': 'Unique Players Count'})
age_working

# Calculating result
age_output = age_working[['Purchase Count', 'Total Purchase Value']]
age_output['Average Purchase Price'] = age_working['Total Purchase Value']/age_working['Purchase Count']
age_output['Avg Total Purchase per Person'] = age_working['Total Purchase Value']/age_working['Unique Players Count']

# Formating results
for col in ['Total Purchase Value', 'Average Purchase Price', 'Avg Total Purchase per Person']:
    age_output[col] = age_output[col].map('${:.2f}'.format)

orders = ['Purchase Count', 'Average Purchase Price', 'Total Purchase Value', 'Avg Total Purchase per Person']

age_output

,Purchase Count,Total Purchase Value,Average Purchase Price,Avg Total Purchase per Person
Age Group,,,,
<10,23,$77.13,$3.35,$4.54
10-14,28,$82.78,$2.96,$3.76
15-19,136,$412.89,$3.04,$3.86
20-24,365,$1114.06,$3.05,$4.32
25-29,101,$293.00,$2.90,$3.81
30-34,73,$214.00,$2.93,$4.12
35-39,41,$147.67,$3.60,$4.76
40+,13,$38.24,$2.94,$3.19


## Top Spenders

* Run basic calculations to obtain the results in the table below


* Create a summary data frame to hold the results


* Sort the total purchase value column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [39]:
# Calculate the number of purchases, total amount spent by each user
# and avg amount  per purchase
players_count = purchase_data.groupby('SN')['Price'].count()
players_total = purchase_data.groupby('SN')['Price'].sum()
players_avg = players_total/players_count

top_spenders = pd.DataFrame({'Purchase Count':players_count,
                                    'Average Purchase Price':players_avg,
                                   'Total Purchase Value':players_total})

# Get top 5 spenders
topFive_spenders = top_spenders.sort_values('Total Purchase Value',ascending=False)[:5]

# Format output

for col in ['Average Purchase Price', 'Total Purchase Value']:
    topFive_spenders[col] = topFive_spenders[col].map('${:.2f}'.format)

topFive_spenders.head()

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


## Most Popular Items

* Retrieve the Item ID, Item Name, and Item Price columns


* Group by Item ID and Item Name. Perform calculations to obtain purchase count, item price, and total purchase value


* Create a summary data frame to hold the results


* Sort the purchase count column in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the summary data frame



In [42]:
# Create a dataframe with item's ID, name and price
items_df = purchase_data[["Item ID","Item Name","Price"]]
# Count how many times each item has been purchased
item_count = items_df.groupby(['Item ID','Item Name']).count()

# Take the mean to obtain the price for each item
item_price = items_df.groupby(['Item ID','Item Name']).mean()

# Merge results into a dataframe
items_merge = pd.merge(item_count,item_price, on=['Item ID','Item Name'])
items_merge.columns = ['Purchase Count', 'Purchase Price']

# Add a column with the total purchase value for each item
items_merge['Total Purchase Value'] = items_merge['Purchase Count']*items_merge['Purchase Price']

# Get top 5 item sorted by purchase count
topFive_by_count = items_merge.sort_values('Purchase Count',ascending=False)[:5]

# Format output
topFive_by_count["Purchase Price"] = topFive_by_count["Purchase Price"].map("${:,.2f}".format)
topFive_by_count["Total Purchase Value"] = topFive_by_count["Total Purchase Value"].map("${:,.2f}".format)

topFive_by_count.head()

,,Purchase Count,Purchase Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16


## Most Profitable Items

* Sort the above table by total purchase value in descending order


* Optional: give the displayed data cleaner formatting


* Display a preview of the data frame



In [43]:
# Get top 5 item sorted by purchase value
topFive_by_value = items_merge.sort_values('Total Purchase Value',ascending=False)[:5]

# Format output
topFive_by_value["Purchase Price"] = topFive_by_value["Purchase Price"].map("${:,.2f}".format)
topFive_by_value["Total Purchase Value"] = topFive_by_value["Total Purchase Value"].map("${:,.2f}".format)

topFive_by_value.head()

,,Purchase Count,Purchase Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
